# Convolutional Neural Network

### Importing the libraries

In [1]:
!pip install --upgrade tensorflow keras Pillow

In [2]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

2021-10-23 00:13:22.262031: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-23 00:13:22.262053: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
tf.__version__

'2.6.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [4]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [5]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [6]:
cnn = tf.keras.models.Sequential()

2021-10-23 00:13:24.180615: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-10-23 00:13:24.180644: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-10-23 00:13:24.180669: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (felipe-300E5M-300E5L): /proc/driver/nvidia/version does not exist
2021-10-23 00:13:24.180887: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Step 1 - Convolution

In [7]:
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='tanh', input_shape=[64, 64, 3]))

### Step 2 - Pooling

In [8]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [9]:
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Adding a third convolutional layer

In [10]:
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='softmax'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [11]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [12]:
cnn.add(tf.keras.layers.Dense(units=128, activation='selu'))

### Step 5 - Output Layer

In [13]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [14]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [15]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

2021-10-23 00:13:25.420789: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/25
250/250 [==============================] - 73s 287ms/step - loss: 0.6994 - accuracy: 0.5054 - val_loss: 0.7411 - val_accuracy: 0.5005
Epoch 2/25
250/250 [==============================] - 65s 258ms/step - loss: 0.6749 - accuracy: 0.5659 - val_loss: 0.6030 - val_accuracy: 0.6725
Epoch 3/25
250/250 [==============================] - 64s 257ms/step - loss: 0.6004 - accuracy: 0.6736 - val_loss: 0.5884 - val_accuracy: 0.6880
Epoch 4/25
250/250 [==============================] - 65s 258ms/step - loss: 0.5611 - accuracy: 0.7063 - val_loss: 0.5475 - val_accuracy: 0.7250
Epoch 5/25
250/250 [==============================] - 65s 261ms/step - loss: 0.5235 - accuracy: 0.7355 - val_loss: 0.5048 - val_accuracy: 0.7630
Epoch 6/25
250/250 [==============================] - 63s 250ms/step - loss: 0.5028 - accuracy: 0.7549 - val_loss: 0.4896 - val_accuracy: 0.7625
Epoch 7/25
250/250 [==============================] - 63s 251ms/step - loss: 0.4819 - accuracy: 0.7692 - val_loss: 0.4479 - val_ac

## Part 4 - Making a single prediction

In [16]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

In [17]:
print(prediction)

cat


## Fim